In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import csv

from scipy.signal import find_peaks
from keras.layers import Input, Dense
from keras.models import Model
from numpy import linalg as LA
import sklearn
from sklearn import metrics


In [7]:

def pad(l, content, width):
    l.extend([content] * (width - len(l)))
    return l

####################################
usr_dir="Data_files/"
for user in range(1,23,1):
    usr_file=usr_dir+"U"+str(user)+"_LIHF_itd_file.csv"
    
    
    happy_bucket=[]
    sad_bucket=[]
    stressed_bucket=[]
    relax_bucket=[]
    
    print(usr_file)
    dataset=pd.read_csv(usr_file,header=None)
    dataframe=dataset.values
    
    #print("row_number="+str(dataframe.shape[0]))
    
    # value list holds unique id 
    value_list=[]
    for i in range(dataframe.shape[0]):
        value_list.append(dataframe[i][0])
        
    ID_list=list(set(value_list))
    ID_list.sort()
    print("number of unique ID="+str(len(ID_list)))
    start=0
    next_start=0
    # traverse the whole list and add list of ITD values in emotion list
    for i in range(len(ID_list)):
        ID_number=ID_list[i]
        #print("ID_number="+str(ID_number))
        ITD_values=[]
        
        for j in range(start,dataframe.shape[0],1):
            if(dataframe[j][0]==ID_number):
                ITD_values.append(dataframe[j][2])
                next_start=next_start+1
            else:
                start=next_start
                #print(next_start)
                break
        if(dataframe[j-1][4]==2 and len(ITD_values)>3):
             happy_bucket.append(list(ITD_values))
        if(dataframe[j-1][4]==-2 and len(ITD_values)>3):
             sad_bucket.append(list(ITD_values))
        if(dataframe[j-1][4]==1 and len(ITD_values)>3):
             stressed_bucket.append(list(ITD_values))
        if(dataframe[j-1][4]==0 and len(ITD_values)>3):
            relax_bucket.append(list(ITD_values))
        #print("ID_number="+str(ID_number)+","+"length of list="+str(len(ITD_values)))
    print("sum of bucket length="+str(len(happy_bucket)+len(sad_bucket)+len(stressed_bucket)+len(relax_bucket)))
    count_ITD_list=len(happy_bucket)+len(sad_bucket)+len(stressed_bucket)+len(relax_bucket)
    #print(happy_bucket)
    #print(sad_bucket)
    #print(stressed_bucket)
    #print(relax_bucket)
    
    
    
    ##########################################
    #if(len(happy_bucket)!=0):
        #print("happy bucket present")
    #print(happy_bucket[0])  
    #window = signal.boxcar(51)
    #plt.plot(window)
    #plt.title("Boxcar window")
    #plt.ylabel("Amplitude")
    #plt.xlabel("Sample")
    
    
    
    ####################################################
    #signal=np.array(relax_bucket[1], dtype=float)
    #sp = np.fft.rfft(signal)
    #print(sp.shape)
    #n=signal.size
    #print(n)
    #timestep = 0.1
    #freq = np.fft.rfftfreq(n, d=timestep)
    #print(freq.shape)
    #freq = np.fft.fftfreq(happy_bucket[0].shape[-1], d = 1.)
    #f1=plt.figure()
    #plt.plot(freq, np.abs(sp) ** 2)
    #f2 = plt.figure()
    #plt.plot(freq,np.log10(np.abs(sp) ** 2))
    #plt.show()
    #########################################################
    happy_fft=[]
    sad_fft=[]
    stressed_fft=[]
    relaxed_fft=[]
    happy_sp=0
    list_length=[]
    for i in range(len(happy_bucket)):
        print("happy_bucket="+str(i)+","+"length="+str(len(happy_bucket[i])))
        ################## This part for capturing freq ###############################
        signal=np.array(happy_bucket[i],dtype=float)
        #n=signal.size
        #freq=np.fft.fftfreq(n,d=0.1)
        #print("fft freq length="+str(freq.size))
        #happy_fft.append(list(freq))
        ####################################################
        # sp is set of amplitude
        peak_list=[]
        sp=np.abs(np.fft.rfft(signal))
        peaks,_=find_peaks(sp)
        for j in range(len(peaks)):
            peak_list.append(sp[peaks[j]])
            
        print("fft amplitude length="+str(sp.size))
        happy_fft.append(peak_list)
        
        happy_sp=happy_sp+len(peak_list)
        list_length.append(len(peak_list))
    print("happy sp size length="+str(happy_sp))    
     
    sad_sp=0    
    for i in range(len(sad_bucket)):
        #print("sad_bucket="+str(i)+","+"length="+str(len(sad_bucket[i])))
        ###############################
        signal=np.array(sad_bucket[i],dtype=float)
        #n=signal.size
        #freq=np.fft.fftfreq(n,d=0.1)
        #sad_fft.append(list(freq)) 
        ####################################
        peak_list=[]
        sp=np.abs(np.fft.rfft(signal))
        peaks,_=find_peaks(sp)
        for j in range(len(peaks)):
            peak_list.append(sp[peaks[j]])
        
        print("fft signal length="+str(sp.size))
        sad_fft.append(peak_list)   
        sad_sp=sad_sp+len(peak_list)
        list_length.append(len(peak_list))
    print("sad sp size length="+str(sad_sp))     
        
        
    stressed_sp=0    
    for i in range(len(stressed_bucket)):
        #print("stressed_bucket="+str(i)+","+"length="+str(len(stressed_bucket[i])))
        ###########################################
        signal=np.array(stressed_bucket[i],dtype=float)
        #n=signal.size
        #freq=np.fft.fftfreq(n,d=0.1)
        #stressed_fft.append(list(freq)) 
        ###########################################
        #signal=np.array(stressed_bucket[i],dtype=float)
        peak_list=[]
        sp=np.abs(np.fft.rfft(signal))
        peaks,_=find_peaks(sp)
        for j in range(len(peaks)):
            peak_list.append(sp[peaks[j]])
        print("fft signal length="+str(sp.size))
        stressed_fft.append(peak_list) 
        
        stressed_sp=stressed_sp+len(peak_list)
        list_length.append(len(peak_list))
    print("stressed sp size length="+str(stressed_sp))  
    
    relax_sp=0    
    for i in range(len(relax_bucket)):
        #print("relax_bucket="+str(i)+","+"length="+str(len(relax_bucket[i])))
        #####################################
        signal=np.array(relax_bucket[i],dtype=float)
        #n=signal.size
        #freq=np.fft.fftfreq(n,d=0.1)
        #relaxed_fft.append(list(freq))
        
        #####################################
        #signal=np.array(relax_bucket[i],dtype=float)
        peak_list=[]
        sp=np.abs(np.fft.rfft(signal))
        peaks,_=find_peaks(sp)
        for j in range(len(peaks)):
            peak_list.append(sp[peaks[j]])
            
        print("fft signal length="+str(sp.size))
        relaxed_fft.append(peak_list)   
        relax_sp=relax_sp+len(peak_list)
        list_length.append(len(peak_list))
    print("relaxed sp size length="+str(relax_sp))    
        
    print("length of happy bucket="+str(len(happy_bucket)))
    print("length of happy ftt="+str(len(happy_fft)))
    
    print("length of sad bucket="+str(len(sad_bucket)))
    print("length of sad ftt="+str(len(sad_fft)))
    
    print("length of stressed bucket="+str(len(stressed_bucket)))
    print("length of stressed ftt="+str(len(stressed_fft)))
    
    print("length of relax bucket="+str(len(relax_bucket)))
    print("length of relax ftt="+str(len(relaxed_fft)))
    
    #print("average length="+str((happy_sp+sad_sp+stressed_sp+relax_sp)/count_ITD_list) )
    print("length of maximum list="+str(max(list_length)))
    max_length=max(list_length)
    print(type(happy_fft))
    for i in range(len(happy_fft)):
        happy_fft[i]=pad(happy_fft[i],0,max_length)
        # append label
        happy_fft[i].append(2)
        
    for i in range(len(sad_fft)):
        sad_fft[i]=pad(sad_fft[i],0,max_length)
        sad_fft[i].append(-2)
        
    for i in range(len(stressed_fft)):
        stressed_fft[i]=pad(stressed_fft[i],0,max_length)
        stressed_fft[i].append(1)
        
    for i in range(len(relaxed_fft)):
        relaxed_fft[i]=pad(relaxed_fft[i],0,max_length)
        relaxed_fft[i].append(0)
    ######### writting absolute value of fft in a file for further processing ######################################################################    
    raw_file="Raw_data/All_amp/"+"U"+str(user)+"_LIHF_itd_file.csv"
    with open(raw_file,"w") as resultfile:
        writer=csv.writer(resultfile)
        writer.writerows(happy_fft)
        writer.writerows(sad_fft)
        writer.writerows(stressed_fft)
        writer.writerows(relaxed_fft)
        
            
            
    ######################## Auto Encoder ##########################################
    
    
    

Data_files/U1_LIHF_itd_file.csv
number of unique ID=114
sum of bucket length=114
happy_bucket=0,length=319
fft amplitude length=160
happy_bucket=1,length=176
fft amplitude length=89
happy_bucket=2,length=475
fft amplitude length=238
happy_bucket=3,length=281
fft amplitude length=141
happy_bucket=4,length=120
fft amplitude length=61
happy_bucket=5,length=210
fft amplitude length=106
happy_bucket=6,length=517
fft amplitude length=259
happy_bucket=7,length=83
fft amplitude length=42
happy_bucket=8,length=93
fft amplitude length=47
happy_bucket=9,length=75
fft amplitude length=38
happy_bucket=10,length=921
fft amplitude length=461
happy_bucket=11,length=148
fft amplitude length=75
happy_bucket=12,length=68
fft amplitude length=35
happy_bucket=13,length=105
fft amplitude length=53
happy_bucket=14,length=104
fft amplitude length=53
happy_bucket=15,length=69
fft amplitude length=35
happy_bucket=16,length=68
fft amplitude length=35
happy_bucket=17,length=417
fft amplitude length=209
happy_buck

number of unique ID=49
sum of bucket length=49
happy_bucket=0,length=104
fft amplitude length=53
happy_bucket=1,length=162
fft amplitude length=82
happy_bucket=2,length=119
fft amplitude length=60
happy_bucket=3,length=28
fft amplitude length=15
happy_bucket=4,length=79
fft amplitude length=40
happy_bucket=5,length=64
fft amplitude length=33
happy_bucket=6,length=75
fft amplitude length=38
happy_bucket=7,length=103
fft amplitude length=52
happy_bucket=8,length=5
fft amplitude length=3
happy_bucket=9,length=122
fft amplitude length=62
happy_bucket=10,length=110
fft amplitude length=56
happy_bucket=11,length=101
fft amplitude length=51
happy sp size length=171
fft signal length=71
fft signal length=35
fft signal length=43
fft signal length=40
fft signal length=40
fft signal length=52
sad sp size length=90
fft signal length=44
fft signal length=45
stressed sp size length=28
fft signal length=42
fft signal length=53
fft signal length=34
fft signal length=38
fft signal length=42
fft signal 

number of unique ID=54
sum of bucket length=54
happy sp size length=0
fft signal length=51
fft signal length=27
fft signal length=20
fft signal length=6
fft signal length=27
fft signal length=56
fft signal length=22
fft signal length=189
fft signal length=217
fft signal length=46
fft signal length=20
fft signal length=161
fft signal length=6
fft signal length=30
fft signal length=351
fft signal length=58
fft signal length=44
fft signal length=43
fft signal length=56
sad sp size length=460
fft signal length=68
fft signal length=6
fft signal length=37
fft signal length=46
fft signal length=56
fft signal length=201
fft signal length=20
fft signal length=6
fft signal length=164
fft signal length=25
fft signal length=24
fft signal length=19
fft signal length=45
stressed sp size length=258
fft signal length=10
fft signal length=35
fft signal length=42
fft signal length=18
fft signal length=21
fft signal length=9
fft signal length=7
fft signal length=10
fft signal length=25
fft signal length=

number of unique ID=68
sum of bucket length=67
happy_bucket=0,length=1149
fft amplitude length=575
happy_bucket=1,length=444
fft amplitude length=223
happy_bucket=2,length=180
fft amplitude length=91
happy_bucket=3,length=73
fft amplitude length=37
happy_bucket=4,length=50
fft amplitude length=26
happy_bucket=5,length=58
fft amplitude length=30
happy_bucket=6,length=80
fft amplitude length=41
happy_bucket=7,length=27
fft amplitude length=14
happy_bucket=8,length=667
fft amplitude length=334
happy_bucket=9,length=30
fft amplitude length=16
happy_bucket=10,length=17
fft amplitude length=9
happy_bucket=11,length=138
fft amplitude length=70
happy_bucket=12,length=58
fft amplitude length=30
happy_bucket=13,length=220
fft amplitude length=111
happy sp size length=551
fft signal length=117
fft signal length=82
fft signal length=23
sad sp size length=67
fft signal length=18
fft signal length=171
fft signal length=45
fft signal length=105
fft signal length=58
fft signal length=43
fft signal len

Data_files/U17_LIHF_itd_file.csv
number of unique ID=89
sum of bucket length=89
happy sp size length=0
fft signal length=27
fft signal length=58
sad sp size length=35
fft signal length=109
fft signal length=47
fft signal length=87
fft signal length=184
fft signal length=196
fft signal length=221
fft signal length=45
fft signal length=38
fft signal length=151
fft signal length=38
fft signal length=468
fft signal length=111
fft signal length=25
fft signal length=75
fft signal length=63
fft signal length=148
fft signal length=208
fft signal length=38
fft signal length=82
fft signal length=49
fft signal length=41
fft signal length=47
fft signal length=41
fft signal length=94
fft signal length=83
fft signal length=36
fft signal length=77
fft signal length=103
fft signal length=28
fft signal length=48
fft signal length=82
fft signal length=58
fft signal length=129
fft signal length=65
fft signal length=26
fft signal length=84
fft signal length=57
fft signal length=111
fft signal length=1204


number of unique ID=115
sum of bucket length=109
happy_bucket=0,length=107
fft amplitude length=54
happy_bucket=1,length=9
fft amplitude length=5
happy_bucket=2,length=721
fft amplitude length=361
happy_bucket=3,length=328
fft amplitude length=165
happy_bucket=4,length=8
fft amplitude length=5
happy_bucket=5,length=927
fft amplitude length=464
happy_bucket=6,length=10
fft amplitude length=6
happy_bucket=7,length=53
fft amplitude length=27
happy_bucket=8,length=44
fft amplitude length=23
happy_bucket=9,length=664
fft amplitude length=333
happy_bucket=10,length=572
fft amplitude length=287
happy_bucket=11,length=7
fft amplitude length=4
happy_bucket=12,length=587
fft amplitude length=294
happy_bucket=13,length=25
fft amplitude length=13
happy_bucket=14,length=883
fft amplitude length=442
happy_bucket=15,length=708
fft amplitude length=355
happy_bucket=16,length=173
fft amplitude length=87
happy_bucket=17,length=20
fft amplitude length=11
happy_bucket=18,length=25
fft amplitude length=13
